# License Plate Detection with OpenCV

In this project we demonstrate how to use OpenCV only, with traditional computer vision approaches, to perform License Plate Detection (LPD). 

We follow two approaches:

1- __Morphology based approach__: where only morphological transforms are used, along with some rules to detect the LP.

2- __Charater based approach__: in addition to basic morphological approaches, basic char detection, also based on morphology, is used as an extra characteristic of the LP.

Further, the problem of Licence Plate Recognition (LPR), by recognizing the number and digits written, can be addressed by the second approach.

In both approaches, we load HD videos (1080p). Due to the camera position, this is the most effective resolution to detect LP patterns.

In both approaches we merge car detection, using background subtraction, to narrow the search space.

In [1]:

import numpy as np
import cv2
import imutils
import os
from tqdm import tqdm
import matplotlib.pyplot as plt



In [2]:
# Project constants
SCALAR_BLACK = (0.0, 0.0, 0.0)
SCALAR_WHITE = (255.0, 255.0, 255.0)
SCALAR_YELLOW = (0.0, 255.0, 255.0)
SCALAR_GREEN = (0.0, 255.0, 0.0)
SCALAR_RED = (0.0, 0.0, 255.0)

In [3]:
# Helper functions

def plot_img(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

def draw_oriented_bbox(frame, bbox):
    # Oriented
    rotrect = cv2.minAreaRect(bbox)
    center, size, theta = rotrect
    box = cv2.boxPoints(rotrect)
    box = np.int0(box)
    cv2.drawContours(frame, [box], 0, SCALAR_RED, 10)

# Moving object detection (MOD)

In this part, we show how to detect and isolate the car box. 

We use background subtraction. [See this reference](https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/). This is possible due to the fixed camera position.

We can detect bounding rectangle or oriented one. The oriented bbox is not very accurate, and later it turns to be not important for LPD.

In [4]:

def detect_cars(frame, background):
    MIN_AREA = 10000
    cars = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    # compute the absolute difference between the current frame and
    # first frame
    frame_delta = cv2.absdiff(background, gray)
    thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    for k,c in enumerate(cnts):
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < MIN_AREA:
            continue
        car = cv2.boundingRect(c)
        cars.append(car)
    return cars

## Video processing
The `process_video` function takes car of frame processing of the given `video_file`. The output is saved in the location of the output `video_output_file`.

This function can be used to:
- Detect Moving cars.
- Detect LPs within car frames, and plot it back in the original frame.
- Detect LPs in the big frame directly.

In [5]:
def make_1080p(cap):
    cap.set(3, 1920)
    cap.set(4, 1080)

def make_720p(cap):
    cap.set(3, 1280)
    cap.set(4, 720)

def make_480p(cap):
    cap.set(3, 640)
    cap.set(4, 480)

def change_res(width, height):
    cap.set(3, width)
    cap.set(4, height)


def process_video(video_file, # The video path to be processed
                  video_output_file, # The output video file
                  output_video_resolution=(640,480), # The desired output resolution
                  frames_cnt=None, # The desired number of frames to process. If None the whole video is processed.
                  cars_detection=True, # LPD will work on the car cropped image or whole image.
                  show_cars_bbox=0,# 0=dont show, 1: show rect, 2: show oriented bbox.
                  detect_LP_fn=None, # The LPD function.
                  debug=False):

    	

    # Set input capture to 1080p
    cap = cv2.VideoCapture(video_file)
    make_1080p(cap)
    
    # Set the frame count if no passed desired number process the whole video
    if frames_cnt == None: frames_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Prepare the output video file
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(video_output_file,fourcc, 20.0, output_video_resolution)

    # The min detectable car is a 100x100 rectangle
    MIN_AREA = 10000
    
    # Set the back ground frame to nothing
    background = None


    for cnt in tqdm(range(frames_cnt), position=0, leave=True):

        ret, frame = cap.read() 

        if ret:
            if cars_detection:

                if background is None:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    gray = cv2.GaussianBlur(gray, (21, 21), 0)
                    # if the first frame is None, initialize it
                    background = gray
                    continue
                    
                cars = detect_cars(frame, background)
                for car in cars:
                    
                    (x, y, w, h) = car
                    car = frame[y:y+h,x:x+w,:]
                    if debug: print('Car size', car.shape)                        
                    if detect_LP_fn != None:
                        # Pass the cropped car image to LPD
                        car_LP, LPs = detect_LP(car)
                        # Put back the LP patch in the original frame
                        frame[y:y+h,x:x+w,:] = car_LP

                    if show_cars_bbox == 1: # Just rectangle
                        cv2.rectangle(frame, (x, y), (x + w, y + h), SCALAR_RED, 10)
                    elif show_cars_bbox == 2: # Oriented rectangle
                        draw_oriented_bbox(frame, c)
            
            elif detect_LP_fn != None:
                frame, LPs = detect_LP_fn(frame)
                

            if debug: plot_img(frame)
            out.write(cv2.resize(frame, output_video_resolution))
        else:
            print('no video')
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        

    cap.release()
    out.release()
    print('Video is ready at: ', video_output_file)

In [6]:
video_file = 'dat/detection_test.mp4'
video_output_file = 'dat/cars_detection.mp4'
process_video(video_file, video_output_file, show_cars_bbox=1)

100%|██████████| 2171/2171 [01:02<00:00, 34.62it/s]

Video is ready at:  dat/cars_detection.mp4


# Morphology based approach

# Character based approach

The main approach 

# Video production

## Morphology test

## Char based test

## Showing the effect of MOD
### With MOD

### Withoud MOD

# References:
- https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/
- https://sod.pixlab.io/articles/license-plate-detection.html
- https://github.com/MicrocontrollersAndMore/OpenCV_3_License_Plate_Recognition_Python.git